In [ ]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx

# Load the data
# df = pd.read_csv('your_data.csv')  # Replace with your actual dataset

# Example data preparation (Replace with your actual data columns)
df = pd.DataFrame({
    'T_category': np.random.randint(0, 4, 100),
    'N_category': np.random.randint(0, 4, 100),
    'Age': np.random.randint(30, 80, 100),
    'Performance_status': np.random.randint(0, 3, 100),
    'Total_tumor_dose': np.random.randint(40, 70, 100),
    '2_year_survival': np.random.randint(0, 2, 100)
})

# Define the Bayesian Network structure
model = BayesianNetwork([('T_category', '2_year_survival'),
                         ('N_category', '2_year_survival'),
                         ('Age', '2_year_survival'),
                         ('Performance_status', '2_year_survival'),
                         ('Total_tumor_dose', '2_year_survival')])

# Fit the model using Maximum Likelihood Estimation
model.fit(df, estimator=MaximumLikelihoodEstimator)

# Perform inference
inference = VariableElimination(model)
survival_prob = inference.map_query(variables=['2_year_survival'], evidence={'T_category': 2, 'N_category': 2})

# Print the predicted probability of survival
print("Predicted probability of survival:", survival_prob)

# Convert the Bayesian Network to ONNX format
# Convert to a sklearn-compatible format first
class SklearnWrapper:
    def __init__(self, model):
        self.model = model

    def predict_proba(self, X):
        predictions = []
        for _, row in X.iterrows():
            evidence = row.to_dict()
            pred = self.model.map_query(variables=['2_year_survival'], evidence=evidence)
            prob = pred.get('2_year_survival', 0)
            predictions.append([1 - prob, prob])
        return np.array(predictions)

# Wrap the Bayesian model
sklearn_model = SklearnWrapper(inference)

# Convert to ONNX format
initial_type = [('float_input', FloatTensorType([None, len(df.columns) - 1]))]
onnx_model = convert_sklearn(sklearn_model, initial_types=initial_type)

# Save the ONNX model
with open("bayesian_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("Model has been converted to ONNX format.")
